# Compromized solution to imitate syncronous/asyncronous BO

In [1]:
import numpy as np
import sklearn.gaussian_process as gp
from scipy.stats import norm
from scipy.optimize import minimize
from sklearn.gaussian_process.kernels import RBF, Matern
import time
from acquisitions import EI, UCB
from penalizers import LP, HLP
from objectives import branin
from parallel_bo import sample_next_parallel, acq_parallel

In [3]:
def bayesian_optimization(n_iters, function, bounds, acq_func = UCB, penalizer = HLP, local_L = True, n_processes = 4,
                          X_init = None, n_init = 10, gp_params = None, find_min = True, alpha = 1e-5, epsilon = 1e-7):
    
    
    X_tested = []
    y_tested = []
    n_params = bounds.shape[0]
    
    if X_init is None:
        X_init = np.random.uniform(bounds[:,0], bounds[:,1], (n_init, bounds.shape[0]))
    
    for X in X_init:
        X_tested.append(X)
        y_tested.append(function(X))
    
    # numpy array of y needed for processes
    y_np = np.array(y_tested)
    
    # creating the gaussian process
    if gp_params is not None:
        gaussian_process = gp.GaussianProcessRegressor(**args) 
    
    else:
        kernel = Matern()
        gaussian_process = gp.GaussianProcessRegressor(
            kernel = kernel, alpha = alpha, n_restarts_optimizer = 10, normalize_y = True)
        gaussian_process.fit(X_tested, y_tested)

    # Initializing, iteration counts the iteration number
    iteration = 0
    X_eval = X_eval = np.array([[None] * n_params] * n_processes)
    for i in range(n_processes):
        X_eval[i] = sample_next_parallel(acq_parallel, gaussian_process, np.array(X_eval), y_np, bounds, find_min = find_min,
                                                       acq_func = acq_func, penalizer = penalizer, local_L = local_L)
        
    for i in range(n_iters):
        
        finished = np.random.randint(4)
        print(f'{X_eval[finished]} just finished by worker {finished}.')
        X_tested.append(X_eval[finished])
        y_tested.append(function(X_eval[finished]))
        y_np = np.array(y_tested)
        
        # fit an updated gaussian process
        gaussian_process.fit(X_tested, y_tested)
        
        # then start a new process
        X_eval[finished] = sample_next_parallel(acq_parallel, gaussian_process, X_eval, y_np, bounds, find_min = find_min,
                                                       acq_func = acq_func, penalizer = penalizer, local_L = local_L)

        print(f'{X_eval[finished]} is being evaluated.')
    
        time.sleep(1)

    return X_tested, y_tested



X_tested, y_tested = bayesian_optimization(20, branin, bounds = np.array([[-5, 10], [0, 15]]),
                                               n_init = 16, acq_func = UCB, penalizer = HLP, local_L = True)


best_value = np.min(y_tested)
best_iter = np.argmin(y_tested)
best_X = X_tested[best_iter]
print(best_X, best_value, best_iter)

[-5.0 13.83339725883287] just finished.
Worker 2 finished. 
 [10.0 2.5420560409002655] is being evaluated.
[-5.0 11.740858410957282] just finished.
Worker 0 finished. 
 [-2.4396829995637495 10.880137624518323] is being evaluated.
[-5.0 9.871889432554717] just finished.
Worker 3 finished. 
 [-4.537610922965888 15.0] is being evaluated.
[10.0 2.5420560409002655] just finished.
Worker 2 finished. 
 [-5.0 3.601313953502835] is being evaluated.
[-5.0 3.601313953502835] just finished.
Worker 2 finished. 
 [-5.0 10.02152113616782] is being evaluated.
[-5.0 12.968292664608427] just finished.
Worker 1 finished. 
 [-5.0 0.0] is being evaluated.
[-4.537610922965888 15.0] just finished.
Worker 3 finished. 
 [-1.9965650127042018 5.0420539290584] is being evaluated.
[-5.0 0.0] just finished.
Worker 1 finished. 
 [-4.959359226602665 15.0] is being evaluated.
[-1.9965650127042018 5.0420539290584] just finished.
Worker 3 finished. 
 [-5.0 0.0] is being evaluated.
[-5.0 0.0] just finished.
Worker 3 fini

In [9]:
branin([-3.22696969, 11.47252598])

-0.9951662321003424